In [2]:
import hopsworks
import pandas as pd

In [3]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/314521
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
instruction_set = fs.get_feature_group(name="instructionset", version=4)
df = instruction_set.read()

Finished: Reading data from Hopsworks, using ArrowFlight (1.15s) 


In [5]:
df

,source,page,content
0,01_introduction.pdf,26,A model can also be trained to predict a numbe...
1,02_serverless_ml.pdf,1,Enterprise AI Value Chain\n1 / 54
2,02_serverless_ml.pdf,18,Tabular Data in Pandas\n18 / 54
3,01_introduction.pdf,55,Matrix Addition and Subtraction\n▶The matrices...
4,02_serverless_ml.pdf,9,ML Pipeline Stages - Data Sources\n9 / 54
...,...,...,...
715,09_mlops.pdf,4,MLOps: Automated Testing to Improve ML Product...
716,06_cnn_printable.pdf,16,Vanilla Deep Neural Networks Challenges (1/2)\...
717,04_deep_feedforward_networks_printable.pdf,46,Backpropagation - Backward Pass - Hidden Layer...
718,04_deep_feedforward_networks_printable.pdf,8,Perceptron in TensorFlow\n8 / 58


In [ ]:
df['prompt'] = df['prompt'].str.replace('<s> [INST]', '<s>[INST]')

# add the following question and answer
new_rows = [{'prompt': '<s>[INST] What is the name of the course? [/INST] The name of the course is Scalable Machine Learning and Deep Learning, held at KTH. </s>',
             'questions': 'What is the name of the course?',
             'answers': 'The name of the course is Scalable Machine Learning and Deep Learning, held at KTH.'},
            {'prompt': '<s>[INST] Who is the professor of the course? [/INST] The professor of the course is Jim Dowling. </s>',
             'questions': 'Who is the professor of the course?',
             'answers': 'The professor of the course is Jim Dowling.'}]

new_df = pd.DataFrame(new_rows)
df = pd.concat([df, new_df], ignore_index=True)

# change prompt to text
df = df.rename(columns={'prompt': 'text'})

# save as .csv only the prompt column
df[['text']].to_csv('prompt.csv', index=False)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "mps" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


In [ ]:
generated_ids = model.generate(model_inputs, max_new_tokens=100, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [1]:

from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "matteocirca/Llama-2-13b-chat-hf-smldl"

tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     device_map="auto",
#     torch_dtype='auto'
# ).eval()

# Prompt content: "hi"
messages = [
    {"role": "user", "content": "hi"},
    {"role": "assistant", "content": "Hello! How can I assist you today?"},
    {"role": "user", "content": "I'd like to book a flight from Paris to Berlin."},
    {"role": "assistant", "content": "When do you want to fly?"},
    {"role": "user", "content": "On Friday evening."},
    {"role": "assistant", "content": "Which class do you want to fly?"},
    {"role": "user", "content": "First class."},
    {"role": "assistant", "content": "I'm sorry but we don't have any first class flight available."},
    {"role": "user", "content": "Ok, thanks, goodbye."}
]

input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=False, add_generation_prompt=True, return_tensors='pt')
input_ids
# output_ids = model.generate(input_ids.to('cuda'))
# response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# # Model response: "Hello! How can I assist you today?"
# print(response)


/Users/matteocirca/opt/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


"<s>[INST] hi [/INST] Hello! How can I assist you today? </s><s>[INST] I'd like to book a flight from Paris to Berlin. [/INST] When do you want to fly? </s><s>[INST] On Friday evening. [/INST] Which class do you want to fly? </s><s>[INST] First class. [/INST] I'm sorry but we don't have any first class flight available. </s><s>[INST] Ok, thanks, goodbye. [/INST]"